In [ ]:
import pyedflib
import numpy as np
import pandas as pd
import matplotlib
import pywt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
from numpy import *
from scipy.signal import *
from numpy.fft import * 
from matplotlib import *
from scipy import *
from pylab import *
import pandas
from scipy.fftpack import fft,ifft
import matplotlib.pyplot as plt
import scipy.signal as signal
from scipy import fftpack
import math
import pyedflib
import pyeeg
import scipy
from scipy.signal import *
import io
import struct
import pickle

# Multiple channels to single channel

In [ ]:
# Allows plots to appear directly in the notebook.
%matplotlib inline
#data_path='D:/EEG/EDF/Crumlin/try_separate/identify_AGE/6/train/'
# Load the edf file
f = pyedflib.EdfReader("6-22.edf")
n = f.signals_in_file
signal_labels = f.getSignalLabels()
sigbufs = np.zeros((n, f.getNSamples()[0]))

for i in np.arange(n):
    sigbufs[i, :] = f.readSignal(i)

# and load the data into a DataFrame
df_signals = pd.DataFrame(sigbufs)
df_signals = df_signals.transpose()
df_signals.columns = signal_labels

In [8]:
print('The sample frequency of your data is: ', f.getSampleFrequency(0))

The sample frequency of your data is:  500


## Now convert the edf format to csv file

In [3]:
df_signals.columns=['EEG1','EEG2']
signal_channel=df_signals.EEG2
path='D:/Desktop/Deployment-flask-master/'
signal_channel=pd.DataFrame(signal_channel)
signal_channel.columns=['EEG']
signal_channel=signal_channel[14400000:16200000]
signal_channel.to_csv(path+'EEG_6_22_8-9h.csv', index=False)

# Bio-channel 

In [ ]:
class ChannelHeader:
    def __init__(self,seq,name):
        self.seq = seq
        self.name = name
        self.offset_in_chunk = -1
        self.sample_rate = -1
        #chunk_size means number of elements in one chunk
        self.chunk_size = -1

In [7]:
#coding=utf-8 
# Input
#   edf_filename: full path name of EDF file
# Return
#   data_map: key=channel_name, value=data list
#   cHdrList: header info of each channel
def readRawDataFromEdf(edf_filename):
    with io.open(edf_filename, 'rb') as f:
        cHdrList = readHeader(f)
        #DEBUG
        chunk_size = 0
        for cHdr in cHdrList:
            print ("%s, sample_rate=%d" % (cHdr.name, cHdr.sample_rate))
            chunk_size = chunk_size + cHdr.sample_rate
        print ("total bytes in one chunk: %d" % ((2*chunk_size)))
        data_map = readData(f, cHdrList)
  
    return data_map,cHdrList

#read header information from EDF file stream
#INPUT f: handle to EDF file, caller must gurantee the position is set to START of the file
#RETURN cHdrList: a list, each element represent a channel
def readHeader(f):
    f.seek(243,io.SEEK_CUR)
    chunk_time_len = int(f.read(8))
    #print chunk_time_len
    num_of_channels = int(f.read(4))
    #print num_of_channels
    
    #read num_of_channels*16 byte for names
    cHdrList = []
    for cid in range(num_of_channels):
        channel_name = str(f.read(16)).strip()
        cHdrList.append(ChannelHeader(cid, channel_name))
    
    #Skip data until sample rate
    f.seek(num_of_channels*200, io.SEEK_CUR)
    
    #read sample rate of each channel
    offset = 0
    for cid in range(num_of_channels):
        chunk_size = int(f.read(8))
        cHdrList[cid].offset_in_chunk = offset
        cHdrList[cid].chunk_size = chunk_size
        cHdrList[cid].sample_rate = chunk_size / chunk_time_len

    #skip reserved field
    f.seek(num_of_channels*32 + 1, io.SEEK_CUR)
  
    return cHdrList

#read data from EDF file, each data is a 16 bits integer with sign
#INPUT f: handle to EDF file, caller must gurantee the position is set to START + (num_of_channel + 1 )*256, i.e. the start position of data section
#RETURN data_map: key=channel_name, value=data list
def readData(f, cHdrList):
    #prepare the result data
    data_map = {}
    for cHdr in cHdrList:
        data_map[cHdr.name]=[]
  
    #read each channel 
    eof = False
    chunk_cnt = 0
    while True:
        for cHdr in cHdrList:
        #read data for only one channel in one chunk
            for dataIdx in range(cHdr.chunk_size):
                data_array = f.read(2)
            #DEBUG
            #print data_array.encode('hex')
                if (len(data_array) == 0):
                    eof = True
                    break
                #unpack using little-endian
                data = struct.unpack('<h', data_array)[0]
                data_map[cHdr.name].append(data)
      
            if eof:
                break
    
        chunk_cnt = chunk_cnt +1
        if eof:    
            print ("Chunk_count = %d" % (chunk_cnt))
            break
  
    return data_map  

def writeDataIntoDisk(filename, dataList):
    with open(filename,'w') as f:
        for data in dataList:
            f.write("%d," % data)

def ensureUtf(s):
    try:
        if type(s) == unicode:
            return s.encode('utf8', 'ignore')
    except: 
        return str(s)

In [9]:
filename =ensureUtf("D:/desk/EEG/mouse/Rogerio_data/Lan-data/P24_b_M1.edf")
data_map,cHdrList = readRawDataFromEdf(filename)

b' C3             ', sample_rate=512
b' C4             ', sample_rate=512
b' CZ             ', sample_rate=512
b' F3             ', sample_rate=512
b' F4             ', sample_rate=512
b' F7             ', sample_rate=512
b' F8             ', sample_rate=512
b' FZ             ', sample_rate=512
b' FP1            ', sample_rate=512
b' FP2            ', sample_rate=512
b' FPZ            ', sample_rate=512
b' O1             ', sample_rate=512
b' O2             ', sample_rate=512
b' P3             ', sample_rate=512
b' P4             ', sample_rate=512
b' PZ             ', sample_rate=512
b' T3             ', sample_rate=512
b' T4             ', sample_rate=512
b' T5             ', sample_rate=512
b' T6             ', sample_rate=512
b' AUX1           ', sample_rate=512
b' AUX2           ', sample_rate=512
b' AUX3           ', sample_rate=512
b' AUX4           ', sample_rate=512
b' AUX5           ', sample_rate=512
b' AUX6           ', sample_rate=512
b' AUX7           ', sample_rate=512
b

## Note: sample rate is your sample frequency

In [ ]:
data=pd.DataFrame.from_dict(data_map, orient='index')

In [ ]:
df=data.transpose()
df.columns = ['C3','C4','CZ','F3','F4','F7','F8','FZ','FP1','FP2','FPZ','O1','O2','P3','P4','PZ','T3','T4','T5','T6','AUX1',
              'AUX2','AUX3','AUX4','AUX5','AUX6','AUX7','AUX8','PG1' ,'PG2' ,'A1','A2','EDF Annotations']

In [ ]:
T6_P4=df['T6']-df['P4']
O2_P4=df['O2']-df['P4']
F7_F3=df['F7']-df['F3']
C3_F3=df['C3']-df['F3']
F8_F4=df['F8']-df['F4']
C4_F4=df['C4']-df['F4']
T5_P3=df['T5']-df['P3']
O1_P3=df['O1']-df['P3']

In [ ]:
frames = [T6_P4,O2_P4,F7_F3,C3_F3,F8_F4,C4_F4,T5_P3,O1_P3]
df_bipolar = pd.concat(frames,axis=1)
df_bipolar.columns = ['T6_P4','O2_P4','F7_F3','C3_F3','F8_F4','C4_F4','T5_P3','O1_P3']

In [1]:
import pickle
#读取文件中的内容。注意和通常读取数据的区别之处
df=open('D:/desk/EEG/Rogerio_mice/rogerio_updated_0926/Rogerio_updated_P24.pickle','rb')#注意此处是rb
#此处使用的是load(目标文件)
data=pickle.load(df)
df.close()

In [3]:
df1=data['C3_F3']
df1.columns=['EEG']

In [15]:
path='D:/desk/EDF/Deployment-flask-master/'
f.to_csv(path+'type2_EEG.csv', index=False)

# Matlab to csv

In [ ]:
import scipy.io as sio
data = sio.loadmat('D:/desk/EDF/data/sp1s_aa.mat')

In [ ]:
new_data=data['x_train']

In [15]:
new_data.shape[2]

316

In [6]:
new_data=new_data.T

In [16]:
new_data

array([[[ 41.8, 132. ,  69.4, ...,  85.5,  64.8,  70.9],
        [-10.3,  13. ,  13.2, ...,  11.7,  10.5,  33.6],
        [ 38.1,  24.3,  63.4, ...,  54.7,  65.2, 101.1],
        ...,
        [  7.9, -23. ,   3.2, ...,  19.1,  31.2,  80.7],
        [ 19.2,   0.5,   5.2, ...,  22.6,  41.5,  92.5],
        [ -0.4, -31.8, -19.9, ...,  14.6,  27.9,  85.5]],

       [[ 44.8, 137. ,  57. , ...,  85. ,  90.2,  78.4],
        [ -5.9,   0.4, -16. , ...,   4.3,  44.8,  32.3],
        [ 25.2,   5.2,  21.7, ...,  61.7, 109.2, 110.4],
        ...,
        [ 11.2, -19. ,  15.6, ...,  12.5,  37.7,  77. ],
        [ 33.6,   2.4,  34.4, ...,   1. ,  42.5,  96.8],
        [ 12.7, -21.5,   6.8, ...,  -4.8,  28.5,  88.7]],

       [[ 47.1, 148. ,  76.4, ...,  73.3,  82.7,  71.6],
        [ -3.3,  22.9,  19.4, ...,  -1.1,  30.7,  30.1],
        [ 46. ,  30.6,  67.6, ...,  41.7,  82.7,  96.8],
        ...,
        [ 14.3, -25.3,  16.3, ...,  -4.4,  36. ,  84.8],
        [ 33.8,  -2.3,  33.6, ..., -23. ,  27

In [17]:
import numpy
new_data=numpy.reshape(new_data, (new_data.shape[0]*new_data.shape[2], new_data.shape[1]))

In [19]:
import pandas as pd
new_data=pd.DataFrame(new_data)
new_data

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,41.8,132.0,69.4,53.2,55.6,79.1,121.5,78.5,122.3,71.7,...,50.8,83.6,82.7,64.7,89.1,86.7,33.7,87.1,70.6,28.4
1,94.4,7.1,135.4,80.5,18.7,104.6,52.5,87.0,75.2,93.4,...,81.8,99.4,64.9,124.8,68.4,58.4,98.4,62.4,98.9,110.3
2,50.0,71.8,66.1,81.4,86.8,42.0,64.2,126.9,79.0,63.0,...,22.0,49.7,79.4,87.0,84.0,57.6,113.7,97.9,76.6,41.5
3,52.8,93.8,81.0,54.1,80.4,89.9,94.1,79.0,87.5,130.4,...,72.9,89.8,60.4,57.8,56.9,56.7,56.9,111.6,129.6,38.6
4,66.4,98.4,114.0,40.1,38.4,81.8,91.5,-21.9,80.8,43.1,...,62.9,95.3,85.0,57.0,75.6,76.0,73.0,74.7,65.6,84.2
5,87.6,37.8,67.1,35.1,33.8,78.8,87.8,86.9,66.1,89.4,...,87.7,78.7,55.8,37.0,61.5,95.2,71.8,44.5,61.3,135.0
6,98.6,82.8,61.7,80.0,42.7,62.5,134.5,65.5,85.1,100.1,...,59.9,131.3,9.0,86.6,-42.9,49.3,55.2,73.7,92.4,95.0
7,95.7,56.0,103.7,116.7,106.3,145.0,84.2,56.7,92.0,97.3,...,74.6,86.2,80.6,61.8,70.9,74.6,74.0,45.7,84.2,39.9
8,62.1,56.4,39.2,65.0,127.2,39.8,72.6,27.1,39.1,101.2,...,54.2,23.7,38.2,66.7,96.7,75.8,45.4,95.9,52.8,54.6
9,92.9,34.3,90.8,109.9,68.4,56.5,35.8,43.5,96.6,98.0,...,25.2,88.9,96.1,75.0,66.8,92.6,70.2,49.2,109.9,63.4


In [20]:
new_data.columns=['F3','F1','Fz','F2','F4','FC5','FC3','FC1','FCz','FC2','FC4','FC6','C5','C3',
                  'C1','Cz','C2','C4','C6','CP5','CP3','CP1','CPz','CP2','CP4','CP6','O1','O2']
new_data

,F3,F1,Fz,F2,F4,FC5,FC3,FC1,FCz,FC2,...,C6,CP5,CP3,CP1,CPz,CP2,CP4,CP6,O1,O2
0,41.8,132.0,69.4,53.2,55.6,79.1,121.5,78.5,122.3,71.7,...,50.8,83.6,82.7,64.7,89.1,86.7,33.7,87.1,70.6,28.4
1,94.4,7.1,135.4,80.5,18.7,104.6,52.5,87.0,75.2,93.4,...,81.8,99.4,64.9,124.8,68.4,58.4,98.4,62.4,98.9,110.3
2,50.0,71.8,66.1,81.4,86.8,42.0,64.2,126.9,79.0,63.0,...,22.0,49.7,79.4,87.0,84.0,57.6,113.7,97.9,76.6,41.5
3,52.8,93.8,81.0,54.1,80.4,89.9,94.1,79.0,87.5,130.4,...,72.9,89.8,60.4,57.8,56.9,56.7,56.9,111.6,129.6,38.6
4,66.4,98.4,114.0,40.1,38.4,81.8,91.5,-21.9,80.8,43.1,...,62.9,95.3,85.0,57.0,75.6,76.0,73.0,74.7,65.6,84.2
5,87.6,37.8,67.1,35.1,33.8,78.8,87.8,86.9,66.1,89.4,...,87.7,78.7,55.8,37.0,61.5,95.2,71.8,44.5,61.3,135.0
6,98.6,82.8,61.7,80.0,42.7,62.5,134.5,65.5,85.1,100.1,...,59.9,131.3,9.0,86.6,-42.9,49.3,55.2,73.7,92.4,95.0
7,95.7,56.0,103.7,116.7,106.3,145.0,84.2,56.7,92.0,97.3,...,74.6,86.2,80.6,61.8,70.9,74.6,74.0,45.7,84.2,39.9
8,62.1,56.4,39.2,65.0,127.2,39.8,72.6,27.1,39.1,101.2,...,54.2,23.7,38.2,66.7,96.7,75.8,45.4,95.9,52.8,54.6
9,92.9,34.3,90.8,109.9,68.4,56.5,35.8,43.5,96.6,98.0,...,25.2,88.9,96.1,75.0,66.8,92.6,70.2,49.2,109.9,63.4


# txt to csv 

In [23]:
df = pd.read_fwf('D:/desk/EDF/data/sp1s_aa_train.txt', header=None)

In [24]:
df

,0,1,2,3,4,5,6,7,8,9,...,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400
0,0.0,41.8,44.8,47.1,37.4,40.1,40.6,43.8,43.1,62.8,...,40.1,43.0,46.5,46.3,43.6,36.7,40.7,30.7,40.6,33.1
1,1.0,132.0,137.0,148.0,137.1,153.1,143.1,145.1,140.0,143.1,...,-27.5,-24.5,-31.5,-43.1,-48.7,-53.0,-59.5,-44.6,-31.7,-19.1
2,0.0,69.4,57.0,76.4,52.0,63.0,45.0,63.3,56.8,71.5,...,11.9,22.4,22.0,38.9,26.1,18.6,0.2,15.9,2.2,22.3
3,1.0,53.2,52.5,46.9,51.1,45.1,52.9,57.3,69.1,66.6,...,34.4,31.1,31.0,14.9,31.0,20.8,18.0,4.4,11.7,14.1
4,0.0,55.6,55.8,58.3,62.3,71.5,83.0,93.6,99.1,98.6,...,26.9,23.0,5.6,12.4,8.3,16.8,14.5,27.4,16.4,23.9
5,1.0,79.1,89.2,75.2,82.0,77.8,90.4,80.9,89.5,79.4,...,17.9,28.9,44.1,55.7,63.2,42.7,43.3,42.4,45.7,49.2
6,0.0,121.5,112.2,96.0,97.9,89.9,100.8,83.9,85.5,72.3,...,-7.5,-24.2,-19.1,-19.7,0.2,-13.9,1.6,-1.0,-2.4,-1.0
7,1.0,78.5,80.8,73.5,83.0,98.9,105.9,91.1,79.9,75.1,...,32.6,31.6,38.4,39.8,49.1,50.9,44.8,38.8,30.4,37.7
8,0.0,122.3,114.8,121.4,117.7,115.5,105.3,115.5,115.2,122.0,...,-4.5,14.5,25.4,26.0,2.0,3.4,-8.1,-17.7,-29.7,-28.2
9,0.0,71.7,77.8,72.9,83.1,76.4,82.5,50.9,64.7,56.4,...,-0.6,-10.3,-8.3,-17.0,-23.4,-18.3,-10.8,-11.2,-0.2,14.0


# pickle to csv

In [1]:
import pickle
#读取文件中的内容。注意和通常读取数据的区别之处
df=open('F:/Mouse_EEG/Type_II_Rogerio/M/M4-7_P21-28/M4_7/M7_O1_P3.pickle','rb')#注意此处是rb
#此处使用的是load(目标文件)
data=pickle.load(df)
df.close()

In [2]:
data

,EEG2,label
0,-267.716995,0
1,-414.432438,0
2,-472.841308,0
3,-488.978601,0
4,-412.866791,0
...,...,...
87328995,16.893603,0
87328996,-22.257883,0
87328997,30.557028,0
87328998,-45.403904,0


In [3]:
df1=data['EEG2']

In [4]:
df1.to_csv('mouse1_typeII.csv', index=False)